# Prepare the Data
___

 - Work on copies of the data (keep the original dataset intact).
 - Write functions for all data transformations you apply, for five reasons:
    -  So you can easily prepare the data the next time you get a fresh dataset
    -  So you can apply these transformations in future projects
    -  To clean and prepare the test set
    -  To clean and prepare new data instances once your solution is live
    -  To make it easy to treat your preparation choices as hyperparameters
 - Data cleaning:
    -  Fix or remove outliers (optional).
    -  Fill in missing values (e.g., with zero, mean, median...) or drop their rows (or columns).
 - Feature selection (optional):
   -  Drop the attributes that provide no useful information for the task.
 - Feature engineering, where appropriate:
   -  Discretize continuous features.
   -  Decompose features (e.g., categorical, date/time, etc.).
   -  Add promising transformations of features (e.g., log(x), sqrt(x), x^2, etc.).
   -  Aggregate features into promising new features.
 - Feature scaling:
   -  Standardize or normalize features.

---

In [2]:
# Import Libraries
import pandas as pd
import time
import numpy as np

from sklearn.preprocessing import LabelEncoder

# Set printing in non scientific notation
np.set_printoptions(suppress=True)

In [3]:
# Import Data
start_time = time.time()

business = pd.read_csv("../archive/train_business.csv")
reviews = pd.read_csv("../archive/train_reviews.csv", nrows=100000) # sampling by taking first 1M rows, otherwise need a lot of RAM

end_time = time.time()
print("Took", end_time - start_time, "s")


# Businesses in Las Vegas and currently open business
city = business[(business["city"] == "Las Vegas") & (business["is_open"] == 1)]
las_vegas = city[["business_id", "name", "address", "categories", "stars"]]

Took 3.6074986457824707 s


In [4]:
# Getting just Restaurants
restaurants_only = las_vegas[
    las_vegas["categories"].str.contains("Restaurant.*") == True
].reset_index()

# pull out names and addresses of the restaurants from restaurants_only table
restaurants = restaurants_only[["business_id", "name", "address"]]


# pull out needed columns from review table
df_review = reviews[["user_id", "business_id", "stars", "date"]]


# combine df_review and restaurant table
combined_business_data = pd.merge(df_review, restaurants, on="business_id")
combined_business_data

,user_id,business_id,stars,date,name,address
0,nOTl4aPC4tKHK35T3bNauQ,xVEtGucSRLk5pxxN0t4i6g,5,2012-08-23,"""Eiffel Tower""","""3655 Las Vegas Blvd S"""
1,l0-VQ8nA_oijpKlRB7dkow,xVEtGucSRLk5pxxN0t4i6g,4,2010-02-11,"""Eiffel Tower""","""3655 Las Vegas Blvd S"""
2,JlwWHBFT76iSJe5mWIcZ4A,xVEtGucSRLk5pxxN0t4i6g,4,2015-03-23,"""Eiffel Tower""","""3655 Las Vegas Blvd S"""
3,6YYpJ59jfFCUHkGOpu2NTA,xVEtGucSRLk5pxxN0t4i6g,4,2015-08-07,"""Eiffel Tower""","""3655 Las Vegas Blvd S"""
4,wU3hkI9BFVckpbhd091sVA,xVEtGucSRLk5pxxN0t4i6g,5,2010-08-24,"""Eiffel Tower""","""3655 Las Vegas Blvd S"""
...,...,...,...,...,...,...
11586,oNIaAW9_L7R3IyRrtpU86g,7njZaRjDXi8anBaGlGQu4g,1,2015-09-25,"""The Lodge Cactus""","""4660 W Cactus Ave"""
11587,W4Ha5StQvff-gz81imCR9A,97TVQZZo_2nMDTeWvqfXXg,1,2011-11-15,"""Ciao Ciao Cafe & Bar""","""3500 Las Vegas Blvd S"""
11588,Z0GA65zCMI4ska4iHRr8Bg,U73q64yVTVTKM0RgF8pIag,5,2014-08-02,"""Benny's Smokin BBQ and Brews""","""128 E Fremont St"""
11589,Z0GA65zCMI4ska4iHRr8Bg,KJ6DuC0Jve9hAtzM2SGNOQ,2,2017-03-30,"""Viva El Taco""","""11 N Lamb Blvd"""


Label Encoding Users and Restaurants

In [5]:
# Initialize Label Encoder for Users
user_encode = LabelEncoder()

# Add column with Label Encoded Users
combined_business_data["user"] = user_encode.fit_transform(
    combined_business_data["user_id"].values
)

# Initialize Label Encoder for Restaurants
item_encode = LabelEncoder()

# Add column with Label Encoded Restaurants
combined_business_data["business"] = item_encode.fit_transform(
    combined_business_data["business_id"].values
)
combined_business_data.head(2)

,user_id,business_id,stars,date,name,address,user,business
0,nOTl4aPC4tKHK35T3bNauQ,xVEtGucSRLk5pxxN0t4i6g,5,2012-08-23,"""Eiffel Tower""","""3655 Las Vegas Blvd S""",4189,1920
1,l0-VQ8nA_oijpKlRB7dkow,xVEtGucSRLk5pxxN0t4i6g,4,2010-02-11,"""Eiffel Tower""","""3655 Las Vegas Blvd S""",3989,1920


In [6]:
# Convert Stars to Float
combined_business_data["stars"] = combined_business_data[
    "stars"
].values.astype(np.float32)

In [7]:
# Check
combined_business_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11591 entries, 0 to 11590
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   user_id      11591 non-null  object 
 1   business_id  11591 non-null  object 
 2   stars        11591 non-null  float32
 3   date         11591 non-null  object 
 4   name         11591 non-null  object 
 5   address      11591 non-null  object 
 6   user         11591 non-null  int32  
 7   business     11591 non-null  int32  
dtypes: float32(1), int32(2), object(5)
memory usage: 679.2+ KB


In [8]:
combined_business_data.head(2)

,user_id,business_id,stars,date,name,address,user,business
0,nOTl4aPC4tKHK35T3bNauQ,xVEtGucSRLk5pxxN0t4i6g,5.0,2012-08-23,"""Eiffel Tower""","""3655 Las Vegas Blvd S""",4189,1920
1,l0-VQ8nA_oijpKlRB7dkow,xVEtGucSRLk5pxxN0t4i6g,4.0,2010-02-11,"""Eiffel Tower""","""3655 Las Vegas Blvd S""",3989,1920


In [9]:
# Pickle Data
combined_business_data.to_pickle("combined_business_data.pkl")

In [10]:
# Pickle for Kaggle in protocol 4
combined_business_data.to_pickle("combined_business_data_p4.pkl", protocol=4)